In [1]:
import sys
sys.path.append("/dbfs/grimlock/sri/sidework/")

root_path = "/dbfs/grimlock/sri/sidework/"
root_path = "/Users/sa/CS/stanford/cs229/Project/Neural-Response-Generator-augmented-with-psychological-personalities/"

In [2]:
import pandas as pd
import autoreload
import sys
sys.path.append(root_path + "library/")

%load_ext autoreload
%autoreload 2

# Data preprocessing custom package
import text_preprocessing
from text_preprocessing import Preprocessor

pt = Preprocessor()

# essentials
import numpy as np
from tqdm import tqdm
import pickle
import pickle as pkl
from scipy import sparse
from numpy import asarray
from numpy import savetxt

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /tmp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import numpy as np
import pickle

# To read the files
X_train, X_val, y_train, y_val = np.load(open(root_path + "data/intermediate/MBTI_datasplit_v4_partial_samepost.pkl", "rb"), allow_pickle=True)
data = pickle.load(open(root_path + "data/intermediate/MBTI_data_before_split_v4_partial_samepost.pkl", "rb"))

In [4]:
print ("Training set size: ", X_train.size)
print ("Validation set size: ", X_val.size)

Training set size:  6936
Validation set size:  1735


# Feature Engineering - Vectorize

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(ngram_range=(1,3))

corpus = list(data['post'])
vocabulary = vectorizer.fit_transform(corpus) 

X_train_vec = vectorizer.transform(list(data.loc[X_train]['post']))
X_val_vec = vectorizer.transform(list(data.loc[X_val]['post']))

In [6]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
vocab_red_svd = svd.fit_transform(vocabulary)

In [7]:
print ("Before reduction dim of training and val set: ", X_train_vec.shape[1],"  ", X_val_vec.shape[1])
X_train_vec_red_svd = svd.transform(X_train_vec)
X_val_vec_red_svd = svd.transform(X_val_vec)
print ("After reduction dim of training and val set: ", X_train_vec_red_svd.shape[1]," ", X_val_vec_red_svd.shape[1])

Before reduction dim of training and val set:  6613227    6613227
After reduction dim of training and val set:  100   100


In [ ]:
# # to support MNB - no negative features

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X_train_vec_red_svd = scaler.fit_transform(X_train_vec_red_svd)
# X_val_vec_red_svd = scaler.transform(X_val_vec_red_svd)

In [ ]:
# # Another approach to reduce dimenstions # TO DO: compare


# from sklearn.feature_selection import SelectPercentile, chi2

# X_train_red = SelectPercentile(chi2, percentile=10).fit_transform(X_train_vec, y_train).toarray()
# X_val_red = SelectPercentile(chi2, percentile=10).fit_transform(X_val_vec, y_val).toarray()

In [28]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto')
X_train_sampled, y_train_sampled = smote.fit_sample(X_train_vec_red_svd, y_train)
X_val_sampled, y_val_sampled = smote.fit_sample(X_val_vec_red_svd, y_val)

In [29]:
print ("before resampling: ", X_train_vec_red_svd.shape[0], "  ", X_val_vec_red_svd.shape[0])
print ("after resampling: ", X_train_sampled.shape[0], "  ", X_val_sampled.shape[0])

before resampling:  6936    1735
after resampling:  23440    5856


In [32]:
# store intermediate data results.

np.save(open(root_path + "data/intermediate/MBTI_datasplit_v4_features.pkl", "wb"), np.asarray([X_train_sampled, X_val_sampled, y_train_sampled, y_val_sampled]))

/Users/sa/.venv/dl/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
